In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("/lirneasia/projects/lacuna/data/raw/Leco_data/First 500/1/LOAD_PROFILE_HISTORICAL_READINGS_INVENTORY_17_04_2024.csv")

Number_of_households = 0

first = '/lirneasia/projects/lacuna/data/raw/Leco_data/First 500'
second = '/lirneasia/projects/lacuna/data/raw/Leco_data/Second 500'

In [3]:
data.columns

Index(['SERIAL', 'CUSTOMER_REF', 'TIMESTAMP', 'OBIS', 'DATE', 'TIME',
       'IMPORT_KWH (kWh)', 'EXPORT_KWH (kWh)', 'IMPORT_KVARH (kvarh)',
       'EXPORT_KVARH (kvarh)', 'PHASE_A_INST._CURRENT (A)',
       'PHASE_B_INST._CURRENT (A)', 'PHASE_C_INST._CURRENT (A)',
       'PHASE_A_INST._VOLTAGE (V)', 'PHASE_B_INST._VOLTAGE (V)',
       'PHASE_C_INST._VOLTAGE (V)', 'POWER_FACTOR', 'AVG._IMPORT_KW (kW)',
       'AVG._EXPORT_KW (kW)', 'AVG._IMPORT_KVA (kVA)', 'AVG._EXPORT_KVA (kVA)',
       'AVG._CURRENT (V)', 'AVG._VOLTAGE (V)'],
      dtype='object')

In [4]:
def households_count(root_folder):
    Number_of_households = 0
    cusref = []
    for folder_name in range(1, 11):
        folder_path = os.path.join(root_folder, str(folder_name))
    
        if os.path.exists(folder_path):
            files = os.listdir(folder_path)
        
            for file_name in files:
                if file_name.endswith('.csv'):
                    file_path = os.path.join(folder_path, file_name)
                
                    df = pd.read_csv(file_path, low_memory=False)
                
                    households = df['CUSTOMER_REF']
                    cusref.append(households)
                    unique_count = households.nunique()
                    Number_of_households = Number_of_households + unique_count

        else:
            print(f"Folder '{folder_name}' does not exist.")

    return Number_of_households, cusref


In [5]:

number1, cusref1 = households_count('/lirneasia/projects/lacuna/data/raw/Leco_data/First 500')

number2, cusref2 = households_count('/lirneasia/projects/lacuna/data/raw/Leco_data/Second 500')

final = number1 + number2

In [6]:
print(f"Number of households in all the datasets: {final}")

Number of households in all the datasets: 987


In [7]:
'''Checking whether the households in this dataset are included in the list of households we are a surveying '''

def isin_sampling_pool(root_folder, df_list):

    
    for folder_name in range(1, 11):
        folder_path = os.path.join(root_folder, str(folder_name))
    
        if os.path.exists(folder_path):
            files = os.listdir(folder_path)
        
            for file_name in files:
                if file_name.endswith('.csv'):
                    file_path = os.path.join(folder_path, file_name)
                
                    df = pd.read_csv(file_path, low_memory=False)

                    all_account_nos_included = df['CUSTOMER_REF'].isin(df_list['ACCOUNT_NO']).all()        
                    
        else:
            print(f"Folder '{folder_name}' does not exist.")

    return all_account_nos_included

In [8]:
df_list = pd.read_csv('/lirneasia/projects/lacuna/data/raw/z_other/LA_lacuna_sample.csv')

is_in = isin_sampling_pool('/lirneasia/projects/lacuna/data/raw/Leco_data/Second 500', df_list)

if is_in:
    print("All households in this LECO dataset are inclued in the sample we are surveying.")
else:
    print("Not all households in this LECO dataset are inclued in the sample we are surveying.")

All households in this LECO dataset are inclued in the sample we are surveying.


In [ ]:
'''Identifying households that doesnt have data rows for all the expected date-time combinations'''


# Creating a DataFrame with all expected date-time combinations

date_range = pd.date_range(start="2023-10-01", end="2023-12-14", freq="15T")

all_combinations = pd.DataFrame([(CUSTOMER_REF, date_time_clean) for CUSTOMER_REF in df['CUSTOMER_REF'].unique()  for date_time_clean in date_range], columns=['CUSTOMER_REF', 'date_time_clean'])

merged_df = pd.merge(all_combinations, df, on=['CUSTOMER_REF', 'date_time_clean'], how='left')

In [ ]:
merged_df = merged_df[['CUSTOMER_REF', 'date_time_clean', 'Import kWh (kWh)', 'Export kWh (kWh)',
       'Import kvarh (kvarh)', 'Export kvarh (kvarh)',
       'Phase A Inst. Current (A)', 'Phase B Inst. Current (A)',
       'Phase C Inst. Current (A)', 'Phase A Inst. Voltage (V)',
       'Phase B Inst. Voltage (V)', 'Phase C Inst. Voltage (V)',
       'Phase ABC Power factor', 'Avg. Import kW (kW)', 'Avg. Export kW (kW)',
       'Avg. Import kVA (kVA)', 'Avg. Export kVA (kVA)', 'Avg. Current (V)',
       'Avg. Voltage (V)', 'Inst. Power factor']].copy


In [10]:
data.isna().sum()

SERIAL                            0
CUSTOMER_REF                      0
TIMESTAMP                         0
OBIS                              0
DATE                              0
TIME                              0
IMPORT_KWH (kWh)                  0
EXPORT_KWH (kWh)                  0
IMPORT_KVARH (kvarh)         111672
EXPORT_KVARH (kvarh)         111672
PHASE_A_INST._CURRENT (A)    106496
PHASE_B_INST._CURRENT (A)    111672
PHASE_C_INST._CURRENT (A)    111672
PHASE_A_INST._VOLTAGE (V)    106496
PHASE_B_INST._VOLTAGE (V)    111672
PHASE_C_INST._VOLTAGE (V)    111672
POWER_FACTOR                      0
AVG._IMPORT_KW (kW)           14494
AVG._EXPORT_KW (kW)           14494
AVG._IMPORT_KVA (kVA)        126166
AVG._EXPORT_KVA (kVA)        126166
AVG._CURRENT (V)             144214
AVG._VOLTAGE (V)             144214
dtype: int64

In [9]:
'''Checking for null values in IMPORT_KWH (kWh) and EXPORT_KWH (kWh) '''

def is_null_in_import(root_folder):

    count = 0

    
    for folder_name in range(1, 11):
        folder_path = os.path.join(root_folder, str(folder_name))
    
        if os.path.exists(folder_path):
            files = os.listdir(folder_path)
        
            for file_name in files:
                if file_name.endswith('.csv'):
                    file_path = os.path.join(folder_path, file_name)
                
                    df = pd.read_csv(file_path, low_memory=False)
                    
                    x = data.isna()['IMPORT_KWH (kWh)'].sum()

                    count += x
                    
        else:
            print(f"Folder '{folder_name}' does not exist.")

    return count

In [10]:
count = is_null_in_import(second)

if count == 0:
    print("No null values for IMPORT KWH (KWH)")

else:
    print("There are null values in IMPORT KWH")

No null values for IMPORT KWH (KWH)


In [ ]:
'''No of households with solar panals or have exported electricity'''


# Group by 'CUSTOMER_REF' and check if at least one non-zero value exists in the 'export' column

non_zero_accounts = data.groupby('CUSTOMER_REF')['EXPORT_KWH (kWh)'].any().reset_index()

non_zero_accounts = non_zero_accounts[non_zero_accounts['EXPORT_KWH (kWh)']]



print("No of household that have exported power: ", len(non_zero_accounts))